In [1]:
# Import Statements
import cv2 as cv
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn import metrics
from matplotlib import pyplot as plt
import time
import sys
import multiprocessing
import os
import rospy
from sensor_msgs.msg import Image
from stereo_msgs.msg import DisparityImage
from sensor_msgs.msg import PointCloud2
from nav_msgs.msg import OccupancyGrid as Map
import message_filters
import cv_bridge
import tf
import math
from std_msgs.msg import String

In [2]:
# Initialization Stuff

rospy.init_node('liveproc')

listener = tf.TransformListener()
pub = rospy.Publisher('updated_map', Map, queue_size=10)

occ_map = None
up_occ_map = None
map_resolution = 0.5
origin = None
robot_x = 0
robot_y = 0
map_width = 0

map_height = 0
camera_range = 85/map_resolution
overlay = True
display_robot_yaw = False
display_rect = True
display_map = False
display_info = False
tri_width = 5
dist_error = 0.15

np.set_printoptions(threshold=sys.maxsize)

In [3]:
def createLineIterator(P1, P2, img):
# """
# Produces and array that consists of the coordinates and intensities of
# each pixel in a line between two points

# Parameters:
#     -P1: a numpy array that consists of the coordinate of the first point (x,y)
#     -P2: a numpy array that consists of the coordinate of the second point (x,y)
#     -img: the image being processed

# Returns:
#     -it: a numpy array that consists of the coordinates and intensities of each
#     pixel in the radii (shape: [numPixels, 3], row = [x,y,intensity])     
# """
   #define local variables for readability
    #print img.shape
    image_height = img.shape[0]
    image_width = img.shape[1]
    P1X = P1[0]
    P1Y = P1[1]
    P2X = P2[0]
    P2Y = P2[1]
    
   #difference and absolute difference between points
   #used to calculate slope and relative location between points
    dX = P2X - P1X
    dY = P2Y - P1Y
    dXa = np.abs(dX)
    dYa = np.abs(dY)

    #predefine numpy array for output based on distance between points
    itbuffer = np.empty(shape=(np.maximum(dYa,dXa),3), dtype=np.int32)
    itbuffer.fill(np.nan)

   #Obtain coordinates along the line using a form of Bresenham's algorithm
    negY = P1Y > P2Y
    negX = P1X > P2X
    if P1X == P2X: #vertical line segment
        itbuffer[:,0] = P1X
        if negY:
            itbuffer[:,1] = np.arange(P1Y - 1,P1Y - dYa - 1,-1)
        else:
            itbuffer[:,1] = np.arange(P1Y+1,P1Y+dYa+1)              
    elif P1Y == P2Y: #horizontal line segment
        itbuffer[:,1] = P1Y
        if negX:
            itbuffer[:,0] = np.arange(P1X-1,P1X-dXa-1,-1)
        else:
            itbuffer[:,0] = np.arange(P1X+1,P1X+dXa+1)
    else: #diagonal line segment
        steepSlope = dYa > dXa
        if steepSlope:
            slope = dX.astype(np.float32)/dY.astype(np.float32)
            if negY:
                itbuffer[:,1] = np.arange(P1Y-1,P1Y-dYa-1,-1)
            else:
                itbuffer[:,1] = np.arange(P1Y+1,P1Y+dYa+1)
            itbuffer[:,0] = (slope*(itbuffer[:,1]-P1Y)).astype(np.int) + P1X
        else:
            slope = dY.astype(np.float32)/dX.astype(np.float32)
            if negX:
                itbuffer[:,0] = np.arange(P1X-1,P1X-dXa-1,-1)
            else:
                itbuffer[:,0] = np.arange(P1X+1,P1X+dXa+1)
            itbuffer[:,1] = (slope*(itbuffer[:,0]-P1X)).astype(np.int) + P1Y

   #Remove points outside of image
    colX = itbuffer[:,0]
    colY = itbuffer[:,1]
    itbuffer = itbuffer[(colX >= 0) & (colY >=0) & (colX<image_width) & (colY<image_height)]

   #Get intensities from img ndarray
    itbuffer[:,2] = img[itbuffer[:,1].astype(np.uint),itbuffer[:,0].astype(np.uint)]
    return itbuffer

In [4]:
def disp_rect(rect):
    cv.namedWindow("stereo/left/image_rect_color",cv.WINDOW_NORMAL)
    cv.imshow("stereo/left/image_rect_color", rect)
    cv.waitKey(1)

In [5]:
def disp_map(map_img):
    map_img = np.swapaxes(map_img, 0, 1)
    cv.namedWindow("map_inverted",cv.WINDOW_NORMAL)
    cv.imshow("map_inverted", map_img)
    cv.waitKey(1)

In [6]:
def transform(img):

    global bearings
    global areas
    global tags    
    
    bridge = cv_bridge.CvBridge()
    
    areas = []
    bearings = []
    tags = []
    
    width = img.height
    height = img.width
    
    img = bridge.imgmsg_to_cv2(img, desired_encoding="passthrough")
    img = cv.cvtColor(img, cv.COLOR_RGB2BGR)  
    image = pp(img,width,height)
    img2= dbscan(image, img)

    return img2

def pp(image, width, height):

    image = np.array(image[0:width][0:int(height*0.65)])
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    image[np.logical_not(np.logical_and(image[:,:,1] > 5,
                                        np.logical_or(np.logical_and(image[:,:,0] > 70, image[:,:,0] < 90),
                                        np.logical_or(image[:,:,0] >= 170, image[:,:,0] < 10))))] \
                                        = [0,0,0]
    return image

def dbscan(image, original):

    img = image[:,:,2]
    ret,thresh1 = cv.threshold(img,10,255,cv.THRESH_BINARY)
    nonzero = np.nonzero(thresh1)
    yp = np.array(nonzero[0])
    xp = np.array(nonzero[1])
    X=np.column_stack((xp,yp))
    if len(X) > 0:
        db = DBSCAN(eps=3, min_samples=10).fit(X)
        labels = db.labels_
        array = np.column_stack((X,labels))
        sort = array[np.argsort(array[:, 2])]
        sort = sort[np.logical_not(sort[:,2] == -1)]      
        if len(sort) > 0:
            points = sort[:,[0,1]]
            tb = None
            first_locations = np.delete(np.nonzero(np.r_[1,np.diff(sort[:,2])[:-1]]),0)
            points_split = np.split(points,np.reshape(first_locations,-1))
            for clusters in np.arange(len(points_split)).tolist():
                tb = rectangle(points_split[clusters],image,original)
            original = tb
    return original



In [7]:
def rectangle(mat,image,original):
    
    global overlay
    
    rect = cv.boundingRect(mat)
    min_rect = cv.minAreaRect(mat)

    height = rect[3]
    width = rect[2]
    x = rect[0]
    x = np.int0(x)
    y = rect[1] 
    y = np.int0(y)
    
    tag = 0
    if overlay:
        if width < height < 4*width and width*height > 45:
            tag, original = identify_buoy_overlay(mat,image,original,x,y,width,height,min_rect,tag)
        if tag == 0:
            image[y:y+height][x:x+width]=[0,0,0]
        return original
    else:
        if width < height < 4*width and width*height > 45:
            image = identify_buoy_hsv(mat,image,x,y, width,height,min_rect,tag)
        return image

def identify_buoy_overlay(mat,image,original,x,y,width,height,min_rect,tag):
    
    global bearings
    global areas
    global tags
    
    font = cv.FONT_HERSHEY_SIMPLEX
    color = color_hsv(mat,image)
    if color =='green':
        tag = 2
        cv.putText(original,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(0,128,0),2,cv.LINE_AA)
        tags.append(tag)
    elif color =='red':
        tag = 3
        cv.putText(original,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(0,0,128),2,cv.LINE_AA)
        tags.append(tag)
    else:
        tag = 1
        cv.putText(original,'buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
        tags.append(tag)
    cv.rectangle(original,(x,y),(x+width,y+height),(255,0,255),2)
    bearing = (min_rect[0][0]-640.0)/1280.0*80.0
    cv.putText(original,'bearing: '+ str(round(bearing,2)),(np.int0(x+width/2),np.int0(y+height/2.5)), font, 0.5,(0,0,0),2,cv.LINE_AA)
    bearings.append(bearing)
    area = min_rect[1][0]*min_rect[1][1]
    areas.append(area)
    
    return tag, original

def identify_buoy_hsv(mat,image,x,y,width,height,min_rect,tag):
    
    global bearings
    global areas
    global tags
    
    font = cv.FONT_HERSHEY_SIMPLEX
    color = color_hsv(mat,image)
    
    if color =='green':
        tag = 2
        cv.putText(image,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
        tags.append(tag)
    elif color =='red':
        tag = 3
        cv.putText(image,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
        tags.append(tag)
    else:
        tag = 1
        cv.putText(image,'buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
        tags.append(tag)
    
    cv.rectangle(image,(x,y),(x+width,y+height),(255,0,255),2)
    bearing = (min_rect[0][0]-640.0)/1280.0*80.0
    bearings.append(bearing)
    area = min_rect[1][0]*min_rect[1][1]
    areas.append(area)
    
    return image
    
def color_hsv(mat, image):

    sum_hue = 0
    sum_sat = 0
    sum_val = 0
    mat_add = 0
    
    for x in range(len(mat)):
        if image [mat[x][1]] [mat[x][0]] [2] > 5:
            mat_add = mat_add+1
            sum_val += image [mat[x][1]] [mat[x][0]] [2]
            sum_sat += image [mat[x][1]] [mat[x][0]] [1]
            sum_hue += image [mat[x][1]] [mat[x][0]] [0]
            
    if mat_add > 0:
        sat = sum_sat/mat_add
        val = sum_val/mat_add
        hue = sum_hue/mat_add

        if sat > 5 and val > 5 and (hue < 60 or hue >= 170):
            return 'red'
        if sat > 5 and val > 5 and (70 < hue < 90):
            return 'green'
        else:
            return hue, sat, val
    
    return None

In [8]:
def edgeFunction(a,b,c):
    det = (c[1]-a[1]) * (b[0]-a[0]) - (c[0]-a[0]) * (b[1]-a[1])
    return det

In [9]:
def rasterizeTri(a,b,c,region):
################################################################
# Rasterize a triangle given three points, a region to search,
#  and a dictionary defining map colors. All points are in (x,y)
#  format
#
# Parameters:
# - a,b: 2 points defining the far vertices of the triangle
# - c: The point where the WAM-V is in occ_map
# - d: The point where the WAM-V is in region
# - region: a portion of the occ_map where the triangle may be
# - values: a dict with the point values defining red and green
#     buoys
#
# Returns:
# - new_region: a region color-coded to highlight the triangle
#     and any buoys in the rasterized region
################################################################
    #print(region)
    region[region < 0] = 0
    #print region
    mask = np.array(np.zeros_like(region.copy()))
    updated = cv.fillPoly(mask, [a,b,c], 255)
    print updated
    
    rast_list = []
    start_time = time.time()
    for i in range(int(min(a[0],b[0],c[0])),int(max(a[0],b[0],c[0]))):
        for j in range(int(min(a[1],b[1],c[1])),int(max(a[1],b[1],c[1]))):
            w0 = edgeFunction(b,c,(i,j))
            w1 = edgeFunction(c,a,(i,j))
            w2 = edgeFunction(a,b,(i,j))

            if w0 >= 0 and w1 >= 0 and w2 >= 0:
                rast_list.append([(i),(j),region[i][j]])
#                 if region[i][j] == values.get("green"):
#                     new_region[i][j] = 255
#                 elif region[i][j] == values.get("red"):
#                     new_region[i][j] = 180
#                 else:
#                     new_region[i][j] = 100
    print time.time()-start_time
    return rast_list

In [10]:
def draw_background_objects(map_img,map_np):
    
    global origin
    global robot_x
    global robot_y
    
    o_x = (-origin.x)/map_resolution
    o_y = (-origin.y)/map_resolution
# Color Assignment Section
    map_img[map_np == -1] = [0]    #Change to [255,255,255] for color to [0] for grayscale
    map_img[map_np == 0] = [0]     #Change to [255,255,255] for color to [0] for grayscale
    map_img[map_np == 100] = [100]#Change to [0,0,0] for color to [100] for grayscale
    origin_color = [100]#Change to [0,0,255] for color to [0] for grayscale
    robot_color = [100]#Change to [0,0,255] for color to [0] for grayscale
# Origin (Drawn as a cross)

    map_img[int(o_y)+0][int(o_x)+0] = origin_color
    map_img[int(o_y)+0][int(o_x)+1] = origin_color
    map_img[int(o_y)+0][int(o_x)+2] = origin_color
    map_img[int(o_y)+1][int(o_x)+0] = origin_color
    map_img[int(o_y)+2][int(o_x)+0] = origin_color
    if (-origin.x) != 0:
        map_img[int(o_y)+0][int(o_x)-1] = origin_color
        map_img[int(o_y)+0][int(o_x)-2] = origin_color
    if (-origin.y) != 0:
        map_img[int(o_y)-1][int(o_x)+0] = origin_color
        map_img[int(o_y)-2][int(o_x)+0] = origin_color
# Robot (Drawn as a square)
    map_img[int(robot_y)+0][int(robot_x)+0] = robot_color
    map_img[int(robot_y)+0][int(robot_x)+2] = robot_color
    map_img[int(robot_y)+1][int(robot_x)+2] = robot_color
    map_img[int(robot_y)-1][int(robot_x)+2] = robot_color
    map_img[int(robot_y)+0][int(robot_x)-2] = robot_color
    map_img[int(robot_y)+1][int(robot_x)-2] = robot_color
    map_img[int(robot_y)-1][int(robot_x)-2] = robot_color
    map_img[int(robot_y)+2][int(robot_x)+0] = robot_color
    map_img[int(robot_y)+2][int(robot_x)+1] = robot_color
    map_img[int(robot_y)+2][int(robot_x)-1] = robot_color
    map_img[int(robot_y)-2][int(robot_x)+0] = robot_color
    map_img[int(robot_y)-2][int(robot_x)+1] = robot_color
    map_img[int(robot_y)-2][int(robot_x)-1] = robot_color
    
    return map_img

In [11]:
def draw_ray(map_img,slices):
    
    global display_robot_yaw
    global bearings
    global tags
    global areas
    global robot_x
    global robot_y
    global map_resolution
    global areas
    
    slices_size = len(slices)
    
    if not(display_robot_yaw):
        for i in range(int(slices_size)):
            try:
                arr = slices[i]
                appr_dist_low = (1.0-2.5*dist_error)*map_resolution*2*456.7946397925*math.pow(areas[i],-0.4705300238)
                for ray in arr:
                    if ray[3] > appr_dist_low: 
                        if (ray[2] == -1 or ray[2] == 0):
                            if False: #display true if you want to see slice
                                map_img[ray[1]][ray[0]] = [255,0,0]#Change to [255,0,0] for color to [0] for grayscale
                        if ray[2] == 100:
                            if tags[i] == 2:
                                map_img[ray[1]][ray[0]]=[30]#Change to [0,255,0] for color to 0,[180 for grayscale]
                            if tags[i] == 3: 
                                map_img[ray[1]][ray[0]]=[50]#Change to [0,0,255] for color to 0,[230] for grayscale
            except Exception as ex:
                print ex
    else:
        for arr in slices[-1]:
            map_img[arr[1]][arr[0]] = [0,255,0]
    
    return map_img

In [12]:
def get_mapnbot_info(trans,robot_yaw):

    global map_resolution
    global robot_x
    global robot_y
    global map_width
    global map_height
    global origin
    global occ_map
    global map_np
    
    map_array = occ_map.data
    
    map_width = occ_map.info.width
    map_height = occ_map.info.height
    
    map_np = np.reshape(np.array(map_array),(map_height,map_width))

    robot_x = (trans[0]-origin.x)/map_resolution
    robot_y = (trans[1]-origin.y)/map_resolution
    robot_yaw = (90 - robot_yaw)
    while robot_yaw > 360:
        robot_yaw = robot_yaw - 360
    while robot_yaw < 0:
        robot_yaw = robot_yaw + 360
    
    return map_np, robot_yaw

In [13]:
def insidetriangle((x1,x2,x3),(y1,y2,y3),map_np):

    global robot_x
    global robot_y
    
    xs=np.array((x1,x2,x3),dtype=float)
    ys=np.array((y1,y2,y3),dtype=float)

    # The possible range of coordinates that can be returned
    x_range=np.arange(np.min(xs),np.max(xs)+1)
    y_range=np.arange(np.min(ys),np.max(ys)+1)

    # Set the grid of coordinates on which the triangle lies. The centre of the
    # triangle serves as a criterion for what is inside or outside the triangle.
    X,Y=np.meshgrid( x_range,y_range )
    xc=np.mean(xs)
    yc=np.mean(ys)

    # From the array 'triangle', points that lie outside the triangle will be
    # set to 'False'.
    triangle = np.ones(X.shape,dtype=bool)
    for i in range(3):
        ii=(i+1)%3
        if xs[i]==xs[ii]:
            include = X *(xc-xs[i])/abs(xc-xs[i]) > xs[i] *(xc-xs[i])/abs(xc-xs[i])
        else:
            poly=np.poly1d([(ys[ii]-ys[i])/(xs[ii]-xs[i]),ys[i]-xs[i]*(ys[ii]-ys[i])/(xs[ii]-xs[i])])
            include = Y *(yc-poly(xc))/abs(yc-poly(xc)) > poly(X) *(yc-poly(xc))/abs(yc-poly(xc))
        triangle*=include

    # Output: 2 arrays with the x- and y- coordinates of the points inside the
    # triangle.
    

    x_points = X[triangle]
    y_points = Y[triangle]

    points = np.column_stack((x_points.astype(int), y_points.astype(int)))

    itbuffer = np.empty(shape=((points.size/2),4), dtype=np.int32)
    itbuffer.fill(np.nan)

    itbuffer[:,0] = x_points
    itbuffer[:,1] = y_points
    itbuffer[:,2] = map_np[points[:,1],points[:,0]]

# Get distance values    

    deltax = itbuffer[:,0]-robot_x
    deltay = itbuffer[:,1]-robot_y
    
    deltax = map_resolution*deltax
    deltay = map_resolution*deltay
    
    
    deltax = np.square(deltax)
    deltay = np.square(deltay)

    
    dist_sq = deltax+deltay
    
    dist_sq = np.sqrt(dist_sq)
#     dist_sq = map_resolution*dist_sq
    
    
    itbuffer[:,3] = dist_sq
#     print robot_x,robot_y
#     print np.column_stack((deltax.astype(int),deltay.astype(int)))
    return itbuffer

In [14]:
def get_points(buoy_yaw, index):
    
    global robot_x
    global robot_y
    global camera_range
    global tri_width
    global areas
    global dist_error
    
    appr_dist_up = (1.0+2.0*dist_error)*2*456.7946397925*math.pow(areas[index],-0.4705300238)
    point_deltax = appr_dist_up * math.sin(math.radians(buoy_yaw)) # appr_dist_up
    point_deltay = appr_dist_up * math.cos(math.radians(buoy_yaw)) #appr_dist_up
    per_slope = -1.0/(point_deltay/point_deltax)
    point1 = np.array([round(robot_x), round(robot_y)], dtype=int)
    point2 = np.array([round(robot_x+point_deltax), round(robot_y+point_deltay)], dtype=int)

    
    
    point3_x = (robot_x+point_deltax) + math.sqrt(math.pow(tri_width,2)/(math.pow(per_slope,2)+1))
    point4_x = (robot_x+point_deltax) - math.sqrt(math.pow(tri_width,2)/(math.pow(per_slope,2)+1))

    point3_y = robot_y+point_deltay + per_slope*(point3_x-(robot_x+point_deltax))
    point4_y = robot_y+point_deltay + per_slope*(point4_x-(robot_x+point_deltax))

    point3 = np.array([round(point3_x),round(point3_y)], dtype=int)
    point4 = np.array([round(point4_x),round(point4_y)], dtype=int)
    
    
    return point1,point2,point3,point4

In [15]:
def map_filter(trans, robot_yaw):
    
    global origin
    global robot_x
    global robot_y
    global camera_range
    global occ_map
    global bearings
    global display_robot_yaw
    global tags
    
    map_np, robot_yaw = get_mapnbot_info(trans,robot_yaw) 
    
    slices = []
    i = 0
    while (i < len(bearings)):
        
        
        buoy_yaw = robot_yaw + bearings[i]
        if ((buoy_yaw) < 0):
            buoy_yaw = buoy_yaw + 360
        
        pt1, pt2, pt3, pt4 = get_points(buoy_yaw, i)    
        
#         sliced = createLineIterator(pt1.astype(int),pt2.astype(int),map_np)

        sliced = insidetriangle((pt3[0],pt1[0],pt4[0]),(pt3[1],pt1[1],pt4[1]),map_np)
        slices.append(np.array(sliced).astype(int))
#         print slices
        i+=1
    
    return slices

In [16]:
def get_map(slices):
    
    global camera_range
    global origin
    global robot_x
    global robot_y
    global map_width
    global map_height
    global occ_map
    
    map_np = np.reshape(np.array(occ_map.data),(map_height,map_width))
    
    #Set to 1 when publishing, to 3 for visualization
    
    map_img_z = np.zeros((map_height, map_width, 1))
    
    map_img_b = draw_background_objects(map_img_z,map_np)
    
    map_img_r = draw_ray(map_img_b,slices)
    
    
    
    return map_img_r

In [17]:
def callback(rect):

    global occ_map
    global origin
    global bearings
    global listener 
    global display_rect
    global display_map
    global tags
    global up_occ_map
    global pub
    global map_height
    global map_width
    
    origin = occ_map.info.origin.position
    rect = transform(rect)
    
    if not rospy.is_shutdown():       
        try:
            now = rospy.Time.now()
            start_time = time.time()
            listener.waitForTransform("/odom", "/front_left_camera_link", now, rospy.Duration(0.5))
            print time.time()-start_time
            (trans,rot) = listener.lookupTransform("/odom", "/front_left_camera_link", now)
            euler = tf.transformations.euler_from_quaternion(rot)
            
            pixel_lines = map_filter(trans, math.degrees(euler[2]))
            
            map_image = get_map(pixel_lines)
            
            if True: #set to false if you don't want to publish the updated occupancy map
                up_occ_map.data = np.reshape(map_image,((map_width*map_height),1))
                pub.publish(up_occ_map)
            if display_rect == True:
                disp_rect(rect)
            if display_map == True:
                disp_map(map_image)
        except tf.LookupException as l:
            print l
        except tf.ConnectivityException as c:
            print c
        except tf.ExtrapolationException as e:
            print e
        except Exception as ex:
            print ex

In [18]:
def map_callback(p_map):
    
    global occ_map
    global up_occ_map
    
    occ_map = p_map    
    up_occ_map = p_map

In [19]:
if __name__ == '__main__':   
    proj_map = rospy.Subscriber("projected_map", Map, map_callback)
    rect = rospy.Subscriber("stereo/left/image_rect_color", Image, callback)
    rospy.spin()   

0.0990989208221
0.0507409572601
0.049957036972
0.100821971893
0.0491580963135
0.100437879562
0.0993030071259
0.0999999046326
0.100044965744
0.0495920181274
0.0511178970337
0.0498249530792
0.100296020508
0.0994439125061
0.100278139114
0.0997059345245
0.100188016891
0.0500860214233
0.100723981857
0.100434064865
0.100826025009
0.0496940612793
0.0999798774719
0.0503070354462
0.0499470233917
0.0490260124207
0.0486998558044
0.0500450134277
0.0996949672699
0.0999040603638
0.10050702095
0.100448131561
0.0995528697968
0.0492990016937
0.0504710674286
0.0499980449677
0.0501379966736
0.100528001785
0.100130081177
0.100396871567
0.0991291999817
0.100083112717
0.0497009754181
0.100339889526
0.0488908290863
0.0998740196228
0.099335193634
0.0497500896454
0.0996770858765
0.100787162781
0.0999729633331
0.0999751091003
0.100744009018
0.100221157074
5.19752502441e-05
0.0486090183258
0.0502460002899
0.100203990936
5.10215759277e-05
0.100201845169
0.10088801384
0.0999739170074
0.0499022006989
0.049923896789

Lookup would require extrapolation into the future.  Requested time 1176.630000000 but the latest data is at time 1169.600000000, when looking up transform from frame [front_left_camera_link] to frame [odom]
Lookup would require extrapolation into the future.  Requested time 1177.168000000 but the latest data is at time 1169.600000000, when looking up transform from frame [front_left_camera_link] to frame [odom]
Lookup would require extrapolation into the future.  Requested time 1177.700000000 but the latest data is at time 1169.600000000, when looking up transform from frame [front_left_camera_link] to frame [odom]
Lookup would require extrapolation into the future.  Requested time 1178.237000000 but the latest data is at time 1169.600000000, when looking up transform from frame [front_left_camera_link] to frame [odom]
Lookup would require extrapolation into the future.  Requested time 1178.787000000 but the latest data is at time 1169.600000000, when looking up transform from frame [

Lookup would require extrapolation into the future.  Requested time 1198.155000000 but the latest data is at time 1169.600000000, when looking up transform from frame [front_left_camera_link] to frame [odom]
Lookup would require extrapolation into the future.  Requested time 1198.692000000 but the latest data is at time 1169.600000000, when looking up transform from frame [front_left_camera_link] to frame [odom]
Lookup would require extrapolation into the future.  Requested time 1199.227000000 but the latest data is at time 1169.600000000, when looking up transform from frame [front_left_camera_link] to frame [odom]
Lookup would require extrapolation into the future.  Requested time 1199.761000000 but the latest data is at time 1169.600000000, when looking up transform from frame [front_left_camera_link] to frame [odom]
Lookup would require extrapolation into the future.  Requested time 1200.300000000 but the latest data is at time 1169.600000000, when looking up transform from frame [

Lookup would require extrapolation into the future.  Requested time 1219.656000000 but the latest data is at time 1169.600000000, when looking up transform from frame [front_left_camera_link] to frame [odom]
Lookup would require extrapolation into the future.  Requested time 1220.198000000 but the latest data is at time 1169.600000000, when looking up transform from frame [front_left_camera_link] to frame [odom]
Lookup would require extrapolation into the future.  Requested time 1220.730000000 but the latest data is at time 1169.600000000, when looking up transform from frame [front_left_camera_link] to frame [odom]
Lookup would require extrapolation into the future.  Requested time 1221.271000000 but the latest data is at time 1169.600000000, when looking up transform from frame [front_left_camera_link] to frame [odom]
Lookup would require extrapolation into the future.  Requested time 1221.810000000 but the latest data is at time 1169.600000000, when looking up transform from frame [

Lookup would require extrapolation into the future.  Requested time 1241.217000000 but the latest data is at time 1169.600000000, when looking up transform from frame [front_left_camera_link] to frame [odom]
Lookup would require extrapolation into the future.  Requested time 1241.754000000 but the latest data is at time 1169.600000000, when looking up transform from frame [front_left_camera_link] to frame [odom]
Lookup would require extrapolation into the future.  Requested time 1242.292000000 but the latest data is at time 1169.600000000, when looking up transform from frame [front_left_camera_link] to frame [odom]
Lookup would require extrapolation into the future.  Requested time 1242.827000000 but the latest data is at time 1169.600000000, when looking up transform from frame [front_left_camera_link] to frame [odom]
Lookup would require extrapolation into the future.  Requested time 1243.360000000 but the latest data is at time 1169.600000000, when looking up transform from frame [

...

Lookup would require extrapolation into the future.  Requested time 1781.293000000 but the latest data is at time 1169.600000000, when looking up transform from frame [front_left_camera_link] to frame [odom]
Lookup would require extrapolation into the future.  Requested time 1781.826000000 but the latest data is at time 1169.600000000, when looking up transform from frame [front_left_camera_link] to frame [odom]
Lookup would require extrapolation into the future.  Requested time 1782.364000000 but the latest data is at time 1169.600000000, when looking up transform from frame [front_left_camera_link] to frame [odom]
Lookup would require extrapolation into the future.  Requested time 1782.905000000 but the latest data is at time 1169.600000000, when looking up transform from frame [front_left_camera_link] to frame [odom]
Lookup would require extrapolation into the future.  Requested time 1783.453000000 but the latest data is at time 1169.600000000, when looking up transform from frame [

Lookup would require extrapolation into the future.  Requested time 1802.841000000 but the latest data is at time 1169.600000000, when looking up transform from frame [front_left_camera_link] to frame [odom]
Lookup would require extrapolation into the future.  Requested time 1803.384000000 but the latest data is at time 1169.600000000, when looking up transform from frame [front_left_camera_link] to frame [odom]
Lookup would require extrapolation into the future.  Requested time 1803.931000000 but the latest data is at time 1169.600000000, when looking up transform from frame [front_left_camera_link] to frame [odom]
Lookup would require extrapolation into the future.  Requested time 1804.469000000 but the latest data is at time 1169.600000000, when looking up transform from frame [front_left_camera_link] to frame [odom]
Lookup would require extrapolation into the future.  Requested time 1805.016000000 but the latest data is at time 1169.600000000, when looking up transform from frame [

Lookup would require extrapolation into the future.  Requested time 1824.458000000 but the latest data is at time 1169.600000000, when looking up transform from frame [front_left_camera_link] to frame [odom]
Lookup would require extrapolation into the future.  Requested time 1825.002000000 but the latest data is at time 1169.600000000, when looking up transform from frame [front_left_camera_link] to frame [odom]
Lookup would require extrapolation into the future.  Requested time 1825.538000000 but the latest data is at time 1169.600000000, when looking up transform from frame [front_left_camera_link] to frame [odom]
Lookup would require extrapolation into the future.  Requested time 1826.071000000 but the latest data is at time 1169.600000000, when looking up transform from frame [front_left_camera_link] to frame [odom]
Lookup would require extrapolation into the future.  Requested time 1826.604000000 but the latest data is at time 1169.600000000, when looking up transform from frame [

Lookup would require extrapolation into the future.  Requested time 1845.988000000 but the latest data is at time 1169.600000000, when looking up transform from frame [front_left_camera_link] to frame [odom]
Lookup would require extrapolation into the future.  Requested time 1846.538000000 but the latest data is at time 1169.600000000, when looking up transform from frame [front_left_camera_link] to frame [odom]
Lookup would require extrapolation into the future.  Requested time 1847.078000000 but the latest data is at time 1169.600000000, when looking up transform from frame [front_left_camera_link] to frame [odom]
Lookup would require extrapolation into the future.  Requested time 1847.613000000 but the latest data is at time 1169.600000000, when looking up transform from frame [front_left_camera_link] to frame [odom]
Lookup would require extrapolation into the future.  Requested time 1848.154000000 but the latest data is at time 1169.600000000, when looking up transform from frame [

Lookup would require extrapolation into the future.  Requested time 1867.574000000 but the latest data is at time 1169.600000000, when looking up transform from frame [front_left_camera_link] to frame [odom]
Lookup would require extrapolation into the future.  Requested time 1868.113000000 but the latest data is at time 1169.600000000, when looking up transform from frame [front_left_camera_link] to frame [odom]
Lookup would require extrapolation into the future.  Requested time 1868.648000000 but the latest data is at time 1169.600000000, when looking up transform from frame [front_left_camera_link] to frame [odom]
Lookup would require extrapolation into the future.  Requested time 1869.192000000 but the latest data is at time 1169.600000000, when looking up transform from frame [front_left_camera_link] to frame [odom]
Lookup would require extrapolation into the future.  Requested time 1869.728000000 but the latest data is at time 1169.600000000, when looking up transform from frame [